In [ ]:
import cv2
import torch
import json
import os
from collections import Counter
import matplotlib.pyplot as plt

# --- CONFIGURATION ---
VIDEO_PATH = 'model1/1st.mp4'           # Path to your video file or rtsp link
OUTPUT_DIR = 'output_frames'       # Directory to save annotated frames
ANNOTATED_VIDEO = 'output_video.mp4'  # Output video 
FRAME_INTERVAL = 5                 

# --- SETUP ---
os.makedirs(OUTPUT_DIR, exist_ok=True)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)  #model used

cap = cv2.VideoCapture(VIDEO_PATH)
frame_idx = 0
json_results = []
class_counter = Counter()
frame_diversity = {}
annotated_frames = []

# --- PROCESS VIDEO ---
while True:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % FRAME_INTERVAL == 0:
        # Run detection
        results = model(frame)
        detections = results.xyxy[0].cpu().numpy()  # [x1, y1, x2, y2, conf, class]
        frame_json = {"frame_index": frame_idx, "detections": []}
        classes_in_frame = set()
        for det in detections:
            x1, y1, x2, y2, conf, cls = det
            label = model.names[int(cls)]
            bbox = [float(x1), float(y1), float(x2), float(y2)]
            confidence = float(conf)
            frame_json["detections"].append({
                "class_label": label,
                "bbox": bbox,
                "confidence": confidence
            })
            class_counter[label] += 1
            classes_in_frame.add(label)
            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), 2)
            cv2.putText(frame, f"{label} {confidence:.2f}", (int(x1), int(y1)-10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        frame_diversity[frame_idx] = len(classes_in_frame)
        json_results.append(frame_json)
        # Save annotated frame
        annotated_path = os.path.join(OUTPUT_DIR, f"frame_{frame_idx}.jpg")
        cv2.imwrite(annotated_path, frame)
        annotated_frames.append(frame)
    frame_idx += 1

cap.release()

# --- SAVE JSON RESULTS ---
with open('detection_results.json', 'w') as f:
    json.dump(json_results, f, indent=2)

# --- OBJECT COUNTS PER CLASS ---
print("\nObject Counts Per Class:")
for cls, count in class_counter.items():
    print(f"{cls}: {count}")

# --- FRAME WITH MAXIMUM CLASS DIVERSITY (with error handling) ---
if frame_diversity:
    max_diversity_frame = max(frame_diversity, key=frame_diversity.get)
    print(f"\nFrame with Maximum Class Diversity: {max_diversity_frame} "
          f"({frame_diversity[max_diversity_frame]} unique classes)")
else:
    print("\nNo detections found in any frame. Cannot compute class diversity.")

# --- VISUALIZE OBJECT FREQUENCY ---
if class_counter:
    plt.figure(figsize=(10,6))
    plt.bar(class_counter.keys(), class_counter.values(), color='skyblue')
    plt.xlabel('Object Class')
    plt.ylabel('Frequency')
    plt.title('Object Frequency Across Video')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('object_frequency.png')
    plt.show()
else:
    print("No objects detected, so no bar chart will be generated.")

# --- OPTIONAL: SAVE ANNOTATED VIDEO ---
if annotated_frames:
    height, width, _ = annotated_frames[0].shape
    out = cv2.VideoWriter(ANNOTATED_VIDEO, cv2.VideoWriter_fourcc(*'mp4v'), 15, (width, height))
    for frame in annotated_frames:
        out.write(frame)
    out.release()
    print(f"Annotated video saved as {ANNOTATED_VIDEO}")

print("\nDetection summary complete.")
